<a href="https://colab.research.google.com/github/LorenzoGianassi/Tesi/blob/master/Tesi1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from timeit import default_timer as timer
from torch.utils.data import Subset
import os
import sys
import random
import copy
from torch.backends import cudnn


# associo cuda per lavorare ulla gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# trovo gli indici e salvo una lista per ciascuno
airplane_indices, automobile_indices = [], []
bird_indices, other_indices = [], []
cat_indices, deer_indices = [], []

airplane_idx = trainset.class_to_idx['airplane']
automobile_idx = trainset.class_to_idx['automobile']
bird_idx = trainset.class_to_idx['bird']
cat_idx = trainset.class_to_idx['cat']
deer_idx = trainset.class_to_idx['deer']


def train_split(trainset):
    for i in range(len(trainset)):
        current_class = trainset[i][1]
        if current_class == automobile_idx:
            automobile_indices.append(i)
        elif current_class == deer_idx:
            deer_indices.append(i)
        elif current_class == airplane_idx:
            airplane_indices.append(i)
        elif current_class == bird_idx:
            bird_indices.append(i)
        elif current_class == cat_idx:
            cat_indices.append(i)
        else:
            other_indices.append(i)
    new_trainset1 = torch.utils.data.Subset(trainset, airplane_indices + automobile_indices  + cat_indices + deer_indices + bird_indices)
    new_trainset2 = torch.utils.data.Subset(trainset, other_indices)
    return new_trainset1, new_trainset2

trainset1, trainset2 = train_split(trainset)

  

trainloaderA = torch.utils.data.DataLoader(trainset1, batch_size=128,
                                           shuffle=True, num_workers=0)
trainloaderB = torch.utils.data.DataLoader(trainset2, batch_size=128,
                                           shuffle=True, num_workers=0)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=True, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
#imshow(torchvision.utils.make_grid(images))
# print labels
#print(' '.join('%5s' % classes[labels[j]] for j in range(4)))



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

def train(trainloader,optimizer,criterion,net):
    for epoch in range(10):  # loop over the dataset multiple times
        start = timer()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

           
        end = timer()
        print("Tempo di epoca ",epoch,"uguale a: ", (end - start), "sec")
    print('ALLENAMENTO FINITO')
    print("################################")    
                #####################################################################################
# metodo per eseguire il test
def test(testloader,net):
    # accuracy dell'intero network
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
            100 * correct / total)) 
    # quale classe ha la migliore performance
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))
    print("################################")    
                           #####################################################################################
train(trainloaderA,optimizer,criterion,net)
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

net = Net()
net.load_state_dict(torch.load(PATH))
net.to(device)


# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = net.fc2.in_features
net.fc3_bis = nn.Linear(num_ftrs, 10)

net = net.to(device)

criterion_new = nn.CrossEntropyLoss()
optimizer_new = optim.SGD(net.fc3.parameters(), lr=0.001, momentum=0.9)
train(trainloaderB,optimizer_new,criterion_new, net)
test(testloader, net)

cuda:0
Files already downloaded and verified
Files already downloaded and verified
cuda:0
Tempo di epoca  0 uguale a:  6.59158707000006 sec
Tempo di epoca  1 uguale a:  6.50849094299997 sec
Tempo di epoca  2 uguale a:  6.556964481000023 sec
Tempo di epoca  3 uguale a:  6.558991176000063 sec
Tempo di epoca  4 uguale a:  6.624278916999856 sec
Tempo di epoca  5 uguale a:  6.546896844999992 sec
Tempo di epoca  6 uguale a:  6.651088353999967 sec
Tempo di epoca  7 uguale a:  6.632832970000209 sec
Tempo di epoca  8 uguale a:  6.52833812699987 sec
Tempo di epoca  9 uguale a:  6.73372469800006 sec
ALLENAMENTO FINITO
################################
Tempo di epoca  0 uguale a:  6.527537402999997 sec
Tempo di epoca  1 uguale a:  6.424641733000044 sec
Tempo di epoca  2 uguale a:  6.433066089000022 sec
Tempo di epoca  3 uguale a:  6.463375753000037 sec
Tempo di epoca  4 uguale a:  6.417942339000092 sec
Tempo di epoca  5 uguale a:  6.521118526999999 sec
Tempo di epoca  6 uguale a:  6.463899072000004